EX1: The n-dimensional tensor mastery challenge: Combine the `Head` and `MultiHeadAttention` into one class that processes all the heads in parallel, treating the heads as another batch dimension (answer is in nanoGPT).

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In lecture we implemented multi-head attention by feeding data into several different 'Head' modules in series and then concating the results. We can implement a vectorized version by combining the linear layers of each head into unified linear layers. We just need to be sure to get the dimensions correctly matched up, so that when we create a dimension for each head prior to computing the query-key dot product. I've outlined the shapes of each intermediate tensor in comments.

In [11]:
class SelfAttentionHead(nn.Module):
    def __init__(self, input_channels, output_channels, context_length) -> None:
        super().__init__()
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.key = nn.Linear(input_channels, output_channels, bias=False)
        self.query = nn.Linear(input_channels, output_channels, bias=False)
        self.value = nn.Linear(input_channels, output_channels, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones((context_length, context_length))))
        
    def forward(self, idx):
        B, T, C = idx.shape
        
        # lookup query, key, and value vectors
        # C == input_channels, H == output_channels == head_size
        k = self.key(idx) # (B, T, C) -> (B, T, H)
        q = self.query(idx) # (B, T, C) -> (B, T, H)
        v = self.value(idx) # (B, T, C) -> (B, T, H)

        # compute self attention by taking dot product of query and key
        wei = q @ k.transpose(-2, -1) # (B, T, H) @ (B, H, T) -> (B, T, T)

        wei *= self.output_channels ** -0.5 # scale by sqrt of head size
        
        # apply lower triangular mask to weights
        wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf')) # (B, T, T)

        # apply softmax to get attention weights
        wei = F.softmax(wei, dim=-1) # (B, T, T)

        # apply attention weights to values
        out = wei @ v # (B, T, T) @ (B, T, H) -> (B, T, H)

        return out # (B, T, H)
    

class FeedForward(nn.Module):
    def __init__(self, input_channels, output_channels) -> None:
        super().__init__()

        self.ff = nn.Sequential(
            nn.Linear(input_channels, output_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.ff(x)

class MultiHeadAttention(nn.Module):
    def __init__(self, emb_size, context_length, head_size, num_heads) -> None:
        super().__init__()

        self.heads = nn.ModuleList([
            SelfAttentionHead(emb_size, head_size, context_length) for _ in range(num_heads)
        ])

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)
    

class ResidualTransformerBlock(nn.Module):
    def __init__(self, emb_size, head_size, context_length, num_multi_attn_heads) -> None:
        super().__init__()
        self.attn = MultiHeadAttention(emb_size, context_length, head_size//num_multi_attn_heads, num_heads=num_multi_attn_heads)
        self.ff = FeedForward(head_size, head_size)
        self.norm1 = nn.LayerNorm(head_size)
        self.norm2 = nn.LayerNorm(head_size)

    def forward(self, x):
        x = self.attn(self.norm1(x)) + x # residual
        x = self.ff(self.norm2(x)) + x # residual
        return x
    
class MultiBlockModel(nn.Module):
    def __init__(self, vocab_size, emb_size, head_size, context_length, num_multi_attn_heads, num_blocks) -> None:
        super().__init__()
        assert head_size % num_multi_attn_heads == 0
        self.token_embeddings = nn.Embedding(vocab_size, emb_size)
        self.positional_embeddings = nn.Embedding(context_length, emb_size)
        self.blocks = nn.Sequential(*[ResidualTransformerBlock(emb_size, head_size, context_length, num_multi_attn_heads) for _ in range(num_blocks)], nn.LayerNorm(head_size))
        self.ff = FeedForward(head_size, head_size)
        self.output_layer = nn.Linear(head_size, vocab_size)

        self.context_length = context_length

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # get token and positional embeddings
        t = self.token_embeddings(idx) # (B, T) -> (B, T, C)
        p = self.positional_embeddings(torch.arange(T, device=idx.device)) # (T, C)
        x = t + p # (B, T, C) (broadcasting)

        # pass thru attention head
        x = self.blocks(x) # (B, T, C) -> (B, T, H)
        
        # pass thru feed forward
        x = self.ff(x) # (B, T, H) -> (B, T, H)

        # pass thru output layer
        logits = self.output_layer(x) # (B, T, H) -> (B, T, V)

        
        if targets is not None:
            B, T, V = logits.shape
            logits = logits.view(B*T, V)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_crop = idx[:,-self.context_length:]
            logits, _ = self(idx_crop)
            logits = logits[:,-1,:] # all batches, last timestamp, all channels
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


In [19]:
class MultiHeadAttention(nn.Module):
    def __init__(self, input_channels, output_channels, context_length, num_heads) -> None:
        assert output_channels % num_heads == 0
        super().__init__()
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.head_size = output_channels // num_heads
        self.num_heads = num_heads
        self.key = nn.Linear(input_channels, output_channels, bias=False)
        self.query = nn.Linear(input_channels, output_channels, bias=False)
        self.value = nn.Linear(input_channels, output_channels, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones((context_length, context_length))))
        
    def forward(self, idx):
        B, T, C = idx.shape 
        # B = batches
        # T = timesteps
        # C = input channels
        # Nh = attention heads
        # Hs = head size
        
        q = self.query(idx) # (B, T, C) -> (B, T, Nh * Hs)
        k = self.key(idx) # (B, T, C) -> (B, T, Nh * Hs)
        
        q = q.transpose(-2, -1).view(B, self.num_heads, self.head_size, T).transpose(-2, -1) # (B, T, Nh * Hs) -> (B, Nh, T, Hs)
        k = k.transpose(-2, -1).view(B, self.num_heads, self.head_size, T) # (B, T, Nh * Hs) -> (B, Nh, Hs, T)
        
        w = q @ k # (B, Nh, T, Hs) @ (B, Nh, Hs, T) -> (B, Nh, T, T)
        w = w.masked_fill(self.tril[:T,:T] == 0, float('-inf'))

        w *= self.head_size**-0.5
        w = F.softmax(w, dim=-1)
        
        
        v = self.value(idx) # (B, T, C) -> (B, T, Nh * Hs)
        v = v.transpose(-2, -1).view(B, self.num_heads, self.head_size, T).transpose(-2, -1) # (B, T, Nh * Hs) -> (B, Nh, T, Hs)

        # torch requires a reshape here for reasons I don't fully understand
        attn_out = (w @ v).transpose(-3, -2).reshape(B, T, -1)

        return attn_out
    

# Need to redifine classes that use MultiHeadAttention as well!

class ResidualTransformerBlock(nn.Module):
    def __init__(self, emb_size, head_size, context_length, num_multi_attn_heads) -> None:
        super().__init__()
        self.attn = MultiHeadAttention(emb_size, head_size, context_length, num_heads=num_multi_attn_heads)
        self.ff = FeedForward(head_size, head_size)
        self.norm1 = nn.LayerNorm(head_size)
        self.norm2 = nn.LayerNorm(head_size)

    def forward(self, x):
        x = self.attn(self.norm1(x)) + x # residual
        x = self.ff(self.norm2(x)) + x # residual
        return x
    
class MultiBlockModel(nn.Module):
    def __init__(self, vocab_size, emb_size, head_size, context_length, num_multi_attn_heads, num_blocks) -> None:
        super().__init__()
        assert head_size % num_multi_attn_heads == 0
        self.token_embeddings = nn.Embedding(vocab_size, emb_size)
        self.positional_embeddings = nn.Embedding(context_length, emb_size)
        self.blocks = nn.Sequential(*[ResidualTransformerBlock(emb_size, head_size, context_length, num_multi_attn_heads) for _ in range(num_blocks)], nn.LayerNorm(head_size))
        self.ff = FeedForward(head_size, head_size)
        self.output_layer = nn.Linear(head_size, vocab_size)

        self.context_length = context_length

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # get token and positional embeddings
        t = self.token_embeddings(idx) # (B, T) -> (B, T, C)
        p = self.positional_embeddings(torch.arange(T, device=idx.device)) # (T, C)
        x = t + p # (B, T, C) (broadcasting)

        # pass thru attention head
        x = self.blocks(x) # (B, T, C) -> (B, T, H)
        
        # pass thru feed forward
        x = self.ff(x) # (B, T, H) -> (B, T, H)

        # pass thru output layer
        logits = self.output_layer(x) # (B, T, H) -> (B, T, V)

        
        if targets is not None:
            B, T, V = logits.shape
            logits = logits.view(B*T, V)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_crop = idx[:,-self.context_length:]
            logits, _ = self(idx_crop)
            logits = logits[:,-1,:] # all batches, last timestamp, all channels
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


Now we can try training a model using this new component. I'm going to import some other code that I won't explain here from the 'lilgpt' module I built.

In [20]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

data = torch.tensor(encode(text), dtype=torch.long, device=device)

spl = int(0.9 * len(data))

train_data = data[:spl]

val_data = data[spl:]

torch.manual_seed(1337)

batch_size = 4
context_length = 8

def get_batch(split, batch_size, context_length):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_length, (batch_size,))
    x = torch.stack([data[i:i+context_length] for i in ix])
    y = torch.stack([data[i+1:i+context_length+1] for i in ix])
    return x, y

def train(model, num_steps, batch_size, context_length, learning_rate=1e-3, optimizer=None, print_every=1000):
    if optimizer is None:
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    model.train()
    losses = []
    for step in range(1, num_steps+1):
        optimizer.zero_grad()
        x, y = get_batch('train', batch_size, context_length)
        logits, loss = model(x, y)
        loss.backward()
        optimizer.step()
        if step % print_every == 0:
            print(f'step {step}: loss {loss.item()}')
        losses.append(loss.item())

    return losses


def plot_ema(losses, gamma=0.99, title='ema'):
    ema = losses[0]
    ema_losses = []
    for i, l in enumerate(losses):
        ema = gamma * ema + (1-gamma) * l
        ema_losses.append(ema)
    plt.plot(ema_losses, label='loss ema', color='red')
    plt.legend()
    plt.show()
    print('final loss (ema):', ema_losses[-1])

def generate_text(model, starting_text=' ', max_new_tokens=100):
    data = torch.tensor(encode(starting_text), dtype=torch.long, device=device).reshape(-1, 1)
    model.eval()
    with torch.no_grad():
        print(starting_text + decode(model.generate(data, max_new_tokens=max_new_tokens)[0].tolist()))
    model.train()

In [21]:
context_length = 128
batch_size = 32
emb_size = 64
multi_block_model = MultiBlockModel(vocab_size, emb_size, emb_size, context_length, num_multi_attn_heads=8, num_blocks=8).to(device)
losses = train(multi_block_model, 10000, batch_size, context_length)
plot_ema(losses)
generate_text(multi_block_model)

KeyboardInterrupt: 